# Lab 01 - Engenharia de Dados com Databricks

Neste laboratório, vamos ler um JSON público contendo dados sobre vacinação contra COVID-19 e transformá-lo, tornando-o apto para uso em BI, ML e IA.

## 1 - O primeiro passo é criar um catálogo e um schema para receber nossas tabelas

In [0]:
%sql
-- TO DO: ALTERE NO NOME DO SCHEMA PARA SUAS INICIAIS
CREATE CATALOG IF NOT EXISTS databricks_day;
CREATE SCHEMA IF NOT EXISTS  juliandro;

## 2 - Vamos usar a lib requests para chamar uma URL que nos devolverá um json com os dados que precisamos

In [0]:
import requests

# Baixar o arquivo JSON
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.json"
response = requests.get(url)
json_data = response.json()

# Criar um DataFrame Spark a partir dos dados JSON usando createDataFrame
df = spark.createDataFrame(json_data)

In [0]:
#TO DO: Exiba a estrutura do Dataframe criado acima

In [0]:
# TO DO: Quantas linhas tem o df?

In [0]:
from pyspark.sql.functions import explode, col

# Explodir o array e criar novas colunas a partir dos campos do array
df_exploded = df.withColumn("data", explode(col("data"))).select(col("country"), col("iso_code"), col("data"))



In [0]:
# TO DO:  Quantas linhas tem o df_exploded?

## 3 - Agora, vamos criar as tabelas bronze e silver

In [0]:
from pyspark.sql.functions import map_keys, explode, concat, lit

# No df_exploded, crie um dataframe com todos os nomes distintos das keys do campo data. adicione como prefixo o literal ",data."
keys_df = df_exploded.withColumn("keys", map_keys(col("data"))).select(explode(col("keys")).alias("key")).distinct()
keys_df = keys_df.select(concat(lit(",data."), col("key")).alias("key_with_prefix"))
display(keys_df)

In [0]:
%sql
--DROP TABLE IF EXISTS databricks_day.juliandro.vaccinations_bronze;

In [0]:
# TO DO: Altere o nome do schema da tabela para "databricks_day.juliandro.vaccinations_bronze"

df_exploded.write.mode("overwrite").saveAsTable("databricks_day.juliandro.vaccinations_bronze")

In [0]:
%sql
-- TO DO: Altere o schema da tabela abaixo e crie a tabela vaccinations_silver com o resultado da query
SELECT
country,
iso_code,
CAST(data.date AS DATE) AS date,
CAST(data.daily_people_vaccinated AS BIGINT) AS daily_people_vaccinated,
CAST(data.daily_people_vaccinated_per_hundred AS DOUBLE) AS daily_people_vaccinated_per_hundred,
CAST(data.daily_vaccinations_per_million AS BIGINT) AS daily_vaccinations_per_million,
CAST(data.daily_vaccinations AS BIGINT) AS daily_vaccinations,
CAST(data.daily_vaccinations_raw AS BIGINT) AS daily_vaccinations_raw,
CAST(data.people_vaccinated AS BIGINT) AS people_vaccinated,
CAST(data.people_vaccinated_per_hundred AS DOUBLE) AS people_vaccinated_per_hundred,
CAST(data.people_fully_vaccinated_per_hundred AS DOUBLE) AS people_fully_vaccinated_per_hundred,
CAST(data.people_fully_vaccinated AS BIGINT) AS people_fully_vaccinated,
CAST(data.total_vaccinations_per_hundred AS DOUBLE) AS total_vaccinations_per_hundred,
CAST(data.total_vaccinations AS BIGINT) AS total_vaccinations,
CAST(data.total_boosters AS BIGINT) AS total_boosters,
CAST(data.total_boosters_per_hundred AS DOUBLE) AS total_boosters_per_hundred
FROM databricks_day.juliandro.vaccinations_bronze

## 4 - Vamos criar as tabelas gold que serão utilizadas no próximo Lab

In [0]:
%sql
-- TO DO: Crie uma tabela gold chamada vaccinations_countries_gold que tenha somente países´. Dica: quando é um país, o iso_code tem 3 letras



In [0]:
%sql
-- TO DO: A partir da silver, crie uma tabela gold chamada vaccinations_income_level_gold com os demais registros. Dica: quando NÃO é um país, o iso_code tem mais que 3 letras.

In [0]:
%sql
-- TO DO: Delete da tabela vaccinations_income_level_gold os registros não relacioanados a renda,

## 5 - Vamos enriquecer nossos dados a partir de um csv com o detalhe das vacinas que foram disponibilizadas para cada país

In [0]:
import requests

# Baixar o conteúdo do CSV
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/locations.csv"
response = requests.get(url)
csv_content = response.text

# Converter para DataFrame Spark
import pandas as pd
from io import StringIO

pdf = pd.read_csv(StringIO(csv_content))
spark_df = spark.createDataFrame(pdf)

# Mostrar o DataFrame
display(spark_df)

## 6 - Para as tabelas com detalhes das vacinas, vamos criar usando a inteligência artificial.


***Use os promps abaixo***

In [0]:
# TO DO: execute cada prompt abaixo em uma célula

In [0]:
# Prompt 1: Crie uma tabela chamada vaccines_bronze com o resultado do spark_df
# Prompt 2: Com base na tabela databricks_day.juliandro.vaccines_bronze, crie a databricks_day.juliandro.vaccines_silver fazendo o split da coluna vaccines pelo caractere ",". Cada vacina deve ser uma linha nova. Mantenha somente os campos location (renomeando para country), iso_code e vaccines
# Prompt 3: Com base na silver, crie uma tabela gold chamada databricks_day.juliandro.vaccines_gold com os campos iso_code e qtd_vaccines que será o resultado da contagem de vacinas de cada país

In [0]:
%sql
-- TO DO: Utilizando SQL, delete das tabelas databricks_day.juliandro.vaccines_silver e databricks_day.juliandro.vaccines_gold os registros que o iso_code tenha mais de 3 letras

## 7 - Navegue pelo catálogo, gere descrição para as colunas, analise a linhagem de dados